In [1]:
import pandas as pd
import plotly.offline as pyo

from evm_sleuth.utils.postgres import PostgresClient
from evm_sleuth.config import settings

client = PostgresClient(settings.postgres)
pyo.init_notebook_mode(connected=True)

from helpers.plotting import daily_supply

## Primary market

In [2]:
query = """
SELECT 
    DATE(datetime) as date,
    SUM(CASE 
        WHEN from_address = '0x0000000000000000000000000000000000000000' 
        THEN amount 
        ELSE 0 
    END) as daily_mint_amount,
    SUM(CASE 
        WHEN to_address = '0x0000000000000000000000000000000000000000' 
        THEN amount 
        ELSE 0 
    END) as daily_burn_amount,
    COUNT(CASE 
        WHEN from_address = '0x0000000000000000000000000000000000000000' 
        THEN 1 
    END) as mint_count,
    COUNT(CASE 
        WHEN to_address = '0x0000000000000000000000000000000000000000' 
        THEN 1 
    END) as burn_count
FROM staging.erc20_transfer
WHERE token_address = '0x323c03c48660fe31186fa82c289b0766d331ce21'
GROUP BY DATE(datetime)
ORDER BY date ASC;
"""
df = pd.read_sql(query, client.sqlalchemy_engine)

daily_supply(df, title="$OPEN").show()

## DEX